In [9]:
# Player Similiarity Program
import pandas as pd
import numpy as np
import polars as pl
from api_scraper import MLB_Scrape

# Set display options to print all columns without truncation
pd.set_option("display.max_columns", None)  # Ensure all columns are displayed
pd.set_option("display.max_rows", None)  # Ensure all rows are displayed
pd.set_option("display.width", None)  # Remove column width limit

In [55]:
ab_flag_dict = {'Walk':0, 'Field Error': 1, 'Caught Stealing Home': 0, 'Strikeout Double Play': 1,
                'Sac Fly Double Play': 0,   'Hit By Pitch': 0, 'Runner Out': 0, 'Triple Play': 1, 
                'Field Out': 1, 'Flyout': 1, 'Bunt Lineout': 1, 'Catcher Interference': 0, 'Fielders Choice': 1,
                'Sac Bunt': 0, 'Pop Out': 1, 'Caught Stealing 2B': 0, 'Fielders Choice Out': 1, 'Stolen Base 2B': 0,
                'Sac Fly': 0, 'Groundout': 1, 'Home Run': 1, 'Caught Stealing 3B': 0, 'Intent Walk': 0, 'Double Play': 1,
                'Grounded Into DP': 1, 'Forceout': 1, 'Lineout': 1, 'Single': 1, 'Triple': 1, 'Wild Pitch': 0, 'Double': 1,
                'Strikeout': 1, 'Bunt Groundout': 0, 'Bunt Pop Out': 0}

ab_events = [event for event, flag in ab_flag_dict.items() if flag == 1]

In [41]:
data = pl.read_csv("2025_data.csv")

batters = pl.read_csv("batter_id.csv")

print(data["event"].unique().to_list())

['Walk', 'Field Error', 'Caught Stealing Home', 'Strikeout Double Play', 'Sac Fly Double Play', 'Hit By Pitch', 'Runner Out', 'Triple Play', 'Field Out', 'Flyout', 'Bunt Lineout', 'Catcher Interference', 'Fielders Choice', 'Sac Bunt', 'Pop Out', 'Caught Stealing 2B', 'Fielders Choice Out', 'Stolen Base 2B', 'Sac Fly', None, 'Groundout', 'Home Run', 'Caught Stealing 3B', 'Intent Walk', 'Double Play', 'Grounded Into DP', 'Forceout', 'Lineout', 'Single', 'Triple', 'Wild Pitch', 'Double', 'Strikeout', 'Bunt Groundout', 'Bunt Pop Out']


In [ ]:
player_data = data.filter(pl.col("batter_name") == "Wyatt Langford")

player_ab_data = player_data.filter(pl.col("event").is_in(ab_events))

num_abs = player_ab_data.height


244.5
